In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import ariel_load
import ariel_model
import copy
import tqdm
kgs.sanity_checks_active = True
kgs.sanity_checks_without_errors = True

local


In [2]:
train_data = kgs.load_all_train_data()
test_data = kgs.load_all_test_data()
len(train_data), len(test_data)

(1100, 1)

In [3]:
# loaders = ariel_load.default_loaders()
# for d in tqdm.tqdm(train_data):
#     dd = copy.deepcopy(d)
#     for t in dd.transits:
#         t.load_to_step(5,dd,loaders)
#     kgs.print_sanity_checks()
model = ariel_model.baseline_model()
data = train_data
model.train(data)
inferred_data = model.infer(data)
kgs.score_metric(inferred_data, data)

Inferring: 100%|████████████████████████████████████████████████████████████████████| 1100/1100 [20:13<00:00,  1.10s/it]


Opt time 217.95796632766724
Score:          0.4216
RMS error FGS:  237.73 ppm
RMS error AIRS: 496.73 ppm


(0.42164738173047883, 0.00023773234146904317, 0.0004967263323658412)

In [4]:
kgs.print_sanity_checks()

simple_t0 [10799.912077835505, 16196.720860299289] [9000.0, 18000] 11
simple_residual_diff_FGS [-1.928185938662914e-05, 0.00010923462462865017] [-0.0001, 0.00012] 12
simple_residual_diff_AIRS [-5.083162438973126e-06, 2.1316825559938808e-05] [-0.0001, 3e-05] 13
